In [1]:
import numpy as np

## Problema 1: 

### Problema nº 5 da 3º lista da Disciplina de Computação Científica Aplicada (EQE-044), pelo professor [Luiz Fernando](https://github.com/lflrsilva?tab=overview&from=2021-07-01&to=2021-07-17). Adaptado.
#### Resolução por [Vitor Alexandre](https://www.linkedin.com/in/vitor-alexandre-santos-da-silva-2895a3217/).

Na condição estacionária, a temperatura em qualquer ponto de uma superfície metálica será a média de temperatura de todos os pontos ao seu redor. Isto pode ser usado para propor um procedimento iterativo para cálculo da distribuição da temperatura em todos os pontos de uma placa.

A Fig. 1 apresenta uma placa quadrada dividida em uma malha de 100 quadrados ou nós. A temperatura nos nós formam uma bidimensional (uma matriz). A temperatura em todas as extremidades devem ser mantidas fixas a 20ºC por um sistema de resfriamento enquanto que a temperatura no nó (3,8) está fixa a 100ºC por exposição a água em ebulição.

![placa metálica](..\assets\placaMetálica.png)

Uma nova estimativa da temperatura $T_{i,j}$ e qualquer nó pode ser calculada pela média das temperaturas em seus nós vizinhos. Ou seja, 

$$ T_{i, j}^{novo} = { 1 \over 4 } (T_{i+1, j} + T_{i-1, j} + T_{i, j+1} + T_{i, j-1})$$

Para determinar a distribuição de temperatura na superfície da placa, uma estimativa inicial deve ser definida para todos os nós. Desta forma, a equação acima é aplicada para cada nó que não possui temperatura fixa e calcula-se uma nova estimativa de temperatura. O processo se repete até não exista variação de temperatura entre as iterações (ou um erro para assumir convergência). Neste ponto, a solução estacionária do problema é obtida. Desenvolva um código para calcular o perfil estacionário de temperatura na placa, assumindo que todas as temperaturas inicialmente estão a 20ºC. A equação deve ser aplicada até que o maior erro absoluto de temperatura entre iterações seja menor que 0,001ºC.

Nota: Você pode achar útil guardar a matriz anterior em alguma variável, para comparar com a matriz nova. Entretanto, a matriz nova é uma matriz gerada a partir do cálculo das novas temperaturas com a matriz velha. E provavelmente ela foi copiada da velha. No Python, você, ao fazer isso, não terá criado duas variáveis diferentes, mas duas variáveis que apontam para o mesmo lugar. Por causa disso, as alterações que você fizer em uma, acontecerá na outra. Para evitar esse problema, guarde a matriz velha numa **deepcopy**.

Ex: 

$[In]matrizInicial = [[1, 2, 3], [4, 5, 6]]$  
$[In]matrizVelha = deepcopy(matrizInicial)$  
$[In]matrizInicial[0][0] = 2$  

$[Out]matrizInicial = [[2, 2, 3], [4, 5, 6]]$

*com deepcopy:*  
$[Out]matrizVelha = [[1, 2, 3], [4, 5, 6]]$   
*sem deepcopy:*  
$[Out]matrizVelha = [[2, 2, 3], [4, 5, 6]]$   




In [2]:
from copy import deepcopy

In [3]:
nLinhas = 10  # Número de linhas na placa
nColunas = 10  # Número de colunas na placa
temperaturaInicial = 20  # Temperatura inicial da placa em 20 °C
tolerancia = 0.001  # Erro absoluto máximo admitido
temperaturas = []  # Futura matriz de temperaturas

def main():
    matrizDummy = CriaMatriz(nLinhas, nColunas)
    matrizTemperaturasIniciais = TemperaturasIniciais(matrizDummy)
    temperaturasEquilibrio = TemperaturasEquilibrio(matrizTemperaturasIniciais)
    return temperaturasEquilibrio

def CriaMatriz(nLinhas, nColunas):
# Preenchendo a lista de temperaturas com nLinhas listas de nColunas valores
    matriz = []
    for i in range(nLinhas):
        matriz.append( list(range(nColunas)) )
    return matriz

def TemperaturasIniciais(matrizTemperaturasIniciais):
    # Criação da condição inicial da matriz de temperaturas
    for i in range(nLinhas):
        for j in range(nColunas):
            if (i == 2 and j == 7):  # Índices começando em zero: (3,8) => (2,7)
                matrizTemperaturasIniciais[i][j] = 100
            else:
                matrizTemperaturasIniciais[i][j] = temperaturaInicial
    return matrizTemperaturasIniciais

def TemperaturasEquilibrio(matrizTemperaturas):
    convergencia = False
    
    while not convergencia:
        tempsMemoria = deepcopy(matrizTemperaturas)  # Armazena a matriz anterior
        erroMaxAtual = 0  # Valor máximo de erro na n-ésima iteração

        for i in range(nLinhas):
            for j in range(nColunas):
                if (not (i == 0 or i == nLinhas - 1 or j == 0 or j == nColunas - 1) and
                    not (i == 2 and j == 7)):
                    matrizTemperaturas[i][j] = 1.0 / 4.0 * (
                        matrizTemperaturas[i+1][j]
                        + matrizTemperaturas[i-1][j]
                        + matrizTemperaturas[i][j+1]
                        + matrizTemperaturas[i][j-1]
                    )
                    erro = abs(tempsMemoria[i][j] - matrizTemperaturas[i][j])

                    if (erro > erroMaxAtual):
                        erroMaxAtual = erro

        if (erroMaxAtual < tolerancia):
            convergencia = True
            
    return matrizTemperaturas

In [4]:
matrizTemperaturasEmEquilibrio = main()

## Problema 2:

A matriz gerada no Problema 1 não está lá muito legível....

Crie um algoritmo para **imprimir** matrizes na tela de um jeito mais bonito e compreensível ao leitor.

Algo assim (sem os *underlines*):

1.4___2.2____3  
1_____4.5____3  
1_____2.0____3.2  

- Use um 'tab' para gerar os espaçamentos dos números de uma mesma linha.
- Arredonde os números para uma casa decimal. 

Dicas: 

- Converta os números para string e concatene as strings com '+'.
- Um tab pode ser adicionado ao print com '\t'.
- Se você achar útil, você pode quebrar a linha com '\n'.
- Você pode arredondar o número $x$ para $n$ casas decimais com $round(x, n)$.

In [5]:
def prettyPrint(matriz, casasDecimais):
    for i in range(len(matriz)):
        linha = ''
        for j in range(len(matriz[i])):
            linha = linha + str(round(matriz[i][j], casasDecimais)) + '\t'
        print(linha)

In [6]:
prettyPrint(matrizTemperaturasEmEquilibrio, 1)

20	20	20	20	20	20	20	20	20	20	
20	20.9	22.0	23.6	26.3	31.0	38.9	48.2	34.1	20	
20	21.6	23.5	26.2	30.7	38.9	56.2	100	48.2	20	
20	21.9	24.1	27.0	31.3	37.7	47.0	56.2	38.9	20	
20	21.9	24.0	26.6	29.8	33.8	37.7	38.9	31.0	20	
20	21.6	23.4	25.4	27.6	29.8	31.3	30.7	26.3	20	
20	21.3	22.6	24.0	25.4	26.6	27.0	26.2	23.6	20	
20	20.9	21.7	22.6	23.4	24.0	24.1	23.5	22.0	20	
20	20.4	20.9	21.3	21.6	21.9	21.9	21.6	20.9	20	
20	20	20	20	20	20	20	20	20	20	


## Problema 3:

Cervejas são um produto fermentado e que até para uma mesma espécie de levedura que a fermente, pode gerar produtos distinguivelmente diferentes. Uma mesma espécie de levedura pode ter diferentes linhagens, que estão mudando a todo tempo, de forma que não é conveniente (ou talvez até impraticável) ficar utilizando a mesma linhagem em duas fábricas. Entretanto, grandes produtores de cerveja, para o consumo em massa, prezam por um produto que seja homogêneo. A cerveja X que você toma no Rio deveria ser a mesma que você toma em Petrópolis, por exemplo.

Uma estratégia para contornar o problema é misturar as cervejas de diferentes locais de produção antes do envaze e distribuição.

Os ésteres produzidos ao longo da fermentação são um dos grandes responsáveis por trazer às cervejas suas características, apesar das baixas concentrações. Três dos ésteres mais comuns são o acetato de etila (A), acetato de isoamila (B), acetato de n-propila (C).

Três fábricas (X, Y e Z) da mesma produtora relativamente próximos alimentam um mesmo tanque de mistura. Os laudos de análises químicas trazem as seguintes informações:

|            | X  | Y    | Z    |
|------------|----|------|------|
| A(%)       | 10 | 50   | 25   |
| B(%)       | 45 | 35   | 60   |
| C(%)       | 45 | 15   | 15   |
| Total(ppm) | 70 | 64.2 | 70.1 |

Os demais ésteres que eventualmente estejam presentes na cerveja estão em concentrações aproximadamente desprezíveis, de forma que A, B e C podem ser aproximados a 100% do total de ésteres.

*ppm é o mesmo que 'mg/L'.*

O volume de cerveja de cada fábrica que entrou no tanque foi de:

|            | X   | Y   | Z    |
|------------|-----|-----|------|
| Volume(L)  |20000|80000|60000 |

Calcule a composição de A, B e C, que sairá do tanque de mistura para envaze. Assuma mistura perfeitamente homogênea.

Um sistema de equações do tipo:

$$
    \begin{cases}
    a_{total} = a_xx + a_yy + a_zz \\[4ex]
    b_{total} = b_xx + b_yy + b_zz \\[4ex]
    c_{total} = c_xx + c_yy + c_zz \\[4ex]
    \end{cases}
$$

Pode ser escrito como:

$$
\large
    \begin{bmatrix}
        a_{total} \\[1ex] 
        b_{total} \\[1ex] 
        c_{total} \\[1ex]
    \end{bmatrix}
    =
    \begin{bmatrix}
        a_x & a_y & a_z \\[1ex] 
        b_x & b_y & b_z \\[1ex] 
        c_x & c_y & c_z \\[1ex]
    \end{bmatrix}

    \begin{bmatrix}
        x\\[1ex] 
        y\\[1ex] 
        z\\[1ex] 
    \end{bmatrix}
$$

Sugestão de resolução:

- No nosso problema, o vetor X, Y e Z será as massas totais de ésteres de cada corrente (concentração total multiplicado pelo volume transportado).
- A matriz quadrada pode ser montada exatamente como segue no modelo, com cada éster representando uma linha.
- A [multiplicação das matrizes](https://en.wikipedia.org/wiki/Matrix_multiplication) pode ser, então feita, para obter a massa de cada éster no tanque.
- A composição de cada um pode ser obtida dividindo a massa de cada um pela massa total de ésteres.

Sugestão de execução:

1. Crie uma função que calcula a massa total de ésteres na corrente. 

    - Encontre as massas totais das correntes X, Y e Z.
    - Guarde os resultados numa lista.  

2. Crie uma função que transpõe uma matriz para transformar um vetor linha (vetor deitado) num vetor coluna (vetor em pé).

    - PS1: vetor é uma matriz unidimensional.
    - PS2: 
        - Isso é um vetor linha => [1, 2, 3]  
        - Isso é um vetor coluna => [[1], [2], [3]]  

3. Use a função que transpõe matrizes para gerar um vetor coluna a partir da lista obtida no passo 1.

4. Crie os vetores linha: 'composicaoX', 'composicaoY', 'composicaoZ'.

    - Junte os três para formar uma matriz onde a composição de X é a linha 1, a de Y a linha 2 e a composicao de Z é a linha 3.
    - Transponha essa matriz para gerar a matriz do exemplo.

5. Crie uma função para multiplicar matrizes. Lembre-se de multiplicar as matrizes na **ordem**.

6. Crie uma função que calcule a massa total de ésteres do tanque.
    - Você pode encontrar isso somando as massas de ésteres em todas as correntes.

7. Crie uma função que calcule a composição de ésteres no tanque, que será a massa individual de cada éster pela massa total deles.



In [36]:
CalculaMassa = lambda concentracao, volume: concentracao * volume

def TranspoeMatriz(matriz):

    nLinhas = len(matriz)
    nColunas = len(matriz[0])
    nLinhasMatrizTransposta = nColunas
    nColunasMatrizTransposta = nLinhas

    matrizTransposta = CriaMatriz(nLinhasMatrizTransposta, nColunasMatrizTransposta)
    
    for i in range(nLinhas):
        for j in range(nColunas):
            matrizTransposta[j][i] = matriz[i][j]
    return matrizTransposta

def MultiplicacaoMatrizes(matrizA, matrizB):
    # A X B = C
    assert len(matrizA[0]) == len(matrizB)

    matrizC = CriaMatriz( len(matrizA), len(matrizB[0]) )
    
    for i in range(len(matrizA)):
        for j in range(len(matrizB[0])):
            for n in range(len(matrizB)):
                matrizC[i][j] += matrizA[i][n] * matrizB[n][j]    
    return matrizC  

def CalculaMassaTotalTanque(correntes):
    massaTotal = 0
    for corrente in correntes:
        massaTotal += corrente
    return massaTotal

def CalculaComposicoesTanque():

    matrizComposicoes = [composicaoX, composicaoY, composicaoZ]
    matrizComposicoesTransposta = TranspoeMatriz(matrizComposicoes)

    correnteX = CalculaMassa(concentracaoTotalX, volumeX)
    correnteY = CalculaMassa(concentracaoTotalY, volumeY)
    correnteZ = CalculaMassa(concentracaoTotalZ, volumeZ)
    correntes = [correnteX, correnteY, correnteZ]

    vetorCorrentes = TranspoeMatriz([correntes]) # note que eu tô forçando a existência de len(matriz[0]), do contrário acusaria um erro.
    massaComponentesNoTanque = MultiplicacaoMatrizes(matrizComposicoesTransposta, vetorCorrentes)
    massaTotalNoTanque = CalculaMassaTotalTanque(correntes)
    composicaoTanque = [massaComponentesNoTanque[i][0]/massaTotalNoTanque for i in range(len(correntes))]
    
    return prettyPrint([composicaoTanque], 2)

In [8]:
concentracaoTotalX = 70 #ppm
concentracaoTotalY = 64.2 #ppm
concentracaoTotalZ = 72.1 #ppm
volumeX = 20e3 #L
volumeY = 80e3 #L
volumeZ = 60e3 #L
composicaoX = [0.1, 0.45, 0.45]
composicaoY = [0.5, 0.35, 0.15]
composicaoZ = [0.25, 0.6, 0.15]

In [9]:
CalculaComposicoesTanque()

0.35	0.46	0.19	


## Problema 4:

Depois de um tempo, uma nova fábrica (W) alimentadora do tanque entrou em operação. As leveduras de W vieram de X, que já havia algum tempo, tinha começado a produzir quantidades não desprezíveis de um outro éster, formato de etila (D). Essas mudanças também se refletiram em D. 

Laudos recentes apontam as seguintes composições:

|            | W    | X  | Y    | Z    |
|------------|------|----|------|------|
| A(%)       | 9    | 7  | 45   | 25   |
| B(%)       | 50   | 45 | 35   | 60   |
| C(%)       | 40   | 45 | 20   | 15   |
| D(%)       | 1    | 3  | 0    | 0    |
| Total(ppm) | 100  | 100| 100  | 100  |

Os demais ésteres que eventualmente estejam presentes na cerveja estão em concentrações aproximadamente desprezíveis, de forma que A, B, C e D podem ser aproximados a 100% do total de ésteres.

Encarregaram o estagiário de colocar os dados de fornecimento na planilha de controle do local onde a mistura ocorria. Só que o estagiário perdeu o relatório que continha o dado dos volumes transportados. Com medo de ser demitido, ele bolou um plano pra resolver isso:

- Ele foi no laboratório de controle de qualidade e pediu uma análise do teor de ésteres, que estava saindo na cerveja envazada.
- Depois ele foi na sala de controle, enquanto os operadores estavam em horário de almoço, e pegou os dados de nível no ínicio da batelada.
- Sem compromisso, ele também tava conversando com o pessoal da operação sobre as dimensões do tanque de mistura.

- Conhecendo o raio interno do tanque e o nível no início do envaze ele determinou o volume de líquido.
- Com as concentrações presentes no laudo, em ppm, ele determinou a massa de cada éster no ínicio da batelada ($a$, $b$, $c$ e $d$ totais).
- Com os laudos que ele tinha das fábricas:
    - Ele usou os teores de éster e, com um algoritmo de Eliminação Gaussiana, e encontrou a massa total de ésteres em cada corrente ($w$, $x$, $y$ e $z$).
    - Com a massa total em cada corrente, ele usou as concentrações totais de ésteres para determinar o volume de cada corrente.

O laudo que ele recebeu do laboratório de controle de qualidade:

|        | Saída |
|--------|-------|
| A(ppm) | 30.6  |
| B(ppm) | 47.5  |
| C(ppm) | 21.6  |
| D(ppm) | 0.3   |

Ele estimou um volume de 170m³ de líquido no tanque de mistura (170.000L).

Esse foi o sistema que ele montou:

$$
    \begin{cases}
    a_{total} = a_ww + a_xx + a_yy + a_zz \\[4ex]
    b_{total} = b_ww + b_xx + b_yy + b_zz \\[4ex]
    c_{total} = c_ww + c_xx + c_yy + c_zz \\[4ex]
    d_{total} = d_ww + d_xx + d_yy + d_zz \\[4ex]
    \end{cases}
$$

$$
\large
    \begin{bmatrix}
        a_{total} \\[1ex] 
        b_{total} \\[1ex] 
        c_{total} \\[1ex] 
        d_{total} \\[1ex] 
    \end{bmatrix}
    =
    \begin{bmatrix}
       a_w & a_x & a_y & a_z \\[1ex] 
       b_w & b_x & b_y & b_z \\[1ex] 
       c_w & c_x & c_y & c_z \\[1ex]
       d_w & d_x & d_y & d_z \\[1ex]
    \end{bmatrix}

    \begin{bmatrix}
        w\\[1ex] 
        x\\[1ex] 
        y\\[1ex] 
        z\\[1ex] 
    \end{bmatrix}
$$

*O problema possui múltiplas resoluções, mas, se você seguir o modelo dado, perceba que os elementos da matriz quadrada são um percentual, igual consta nos laudos de concentrações das fábricas (a 1º tabela da célula).* 

$w$, $x$, $y$ e $z$ terão unidade de massa, porque essa é a unidade do vetor do lado esquerdo e a matriz quadrada é adimensional, visto que seus elementos são razões de composição.

### [Eliminação Gaussiana](https://pt.wikipedia.org/wiki/Elimina%C3%A7%C3%A3o_de_Gauss)

Para escalonar a sua matriz, divida o processo em três funções, como sugere a sequência abaixo:

Seja $L_n$ o [pivô](https://en.wikipedia.org/wiki/Pivot_element) da linha i ($l_n$). Seja $L_j$ um elemento da linha j ( $l_j $ )na mesma coluna de $L_n$. Nós podemos zerar $L_j$, atualizando a $l_j$ com o resultado da seguinte operação:

$$\large l_j = l_j - l_n \cdot {L_j \over L_n}$$

*Pivô, no nosso contexto, é o elemento que está na diagonal principal.*

Seja:   
- Crie uma função que atualize $l_j$ conforme a equação acima.
- Crie uma função que recebe a primeira matriz e dá como saída uma matriz triangular superior (a segunda matriz);  
- Crie uma função que recebe a segunda matriz e dá como saída uma matriz diagonal (a terceira matriz);  
- Crie uma função que recebe a terceira matriz e dá como resultado uma matriz identidade (a quarta matriz);  
- Em todas essas funções, sempre que você atualizar a linha $l_j$ da matriz $M$, você deve atualizar a linha $l_j$ do vetor de concentrações totais.  
    - No final, quando você obtiver a matriz identidade a partir de $M$, o vetor $[W, X, Y, Z]$ será igual ao vetor que foi operado ao longo do processo. 

Dica:
- Use a função que você obteve no Problema 2 para testar seu código;
- Use esse [site](https://matrixcalc.org/en/slu.html#solve-using-Gaussian-elimination%28%7B%7B0%2e01,0%2e03,0%2e02,0%2e01,4046000%2e0%7D,%7B0%2e09,0%2e07,0%2e48,0%2e24,5474000%2e0%7D,%7B0%2e5,0%2e45,0%2e35,0%2e6,2261000%2e0%7D,%7B0%2e4,0%2e45,0%2e15,0%2e15,238000%2e0%7D%7D%29) para se familiarizar com a Eliminação Gaussiana.

$$
M =
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14}\\
a_{21} & a_{22} & a_{23} & a_{24}\\
a_{31} & a_{32} & a_{33} & a_{34}\\
a_{41} & a_{42} & a_{43} & a_{44}\\
\end{bmatrix}
$$

$$
M =
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14} &\\
0 & b_{22} & b_{23} & b_{24} &\\
0 & 0 & b_{33} & b_{34} &\\
0 & 0 & 0 & b_{44} &\\
\end{bmatrix}
$$

$$
M =
\begin{bmatrix}
a_{11} & 0 & 0 & 0 & \\
0 & b_{22} & 0 & 0 & \\
0 & 0 & b_{33} & 0 & \\
0 & 0 & 0 & b_{44} & \\
\end{bmatrix}
$$

$$
M =
\begin{bmatrix}
1 & 0 & 0 & 0 & \\
0 & 1 & 0 & 0 & \\
0 & 0 & 1 & 0 & \\
0 & 0 & 0 & 1 & \\
\end{bmatrix}
$$

In [127]:
def MudaLinha (li, lj, razaoPivotamento): 
    # 'lj' é a linha a ser modificada, 'li' a modificadora e 'i' é o elemento a ser zerado
    li_modificador = []
    
    for elemento in li:
        elemento = elemento * razaoPivotamento
        li_modificador.append(elemento)    

    for i in range(len(lj)):
        lj[i] = lj[i] - li_modificador[i]
        
    return lj

def EscalonaInferior (matrizASerEscalonada, vetorResposta): 
    #ela vai escalonar a matriz até zerar a diagonal de baixo
    for n in range(len(matrizASerEscalonada)): #vai passar p/ próximo pivô
        l = 1
        while (l + n) < (len(matrizASerEscalonada)): # vai zerar todos os termos de uma coluna que estão abaixo da diagonal
            razaoPivotamento = matrizASerEscalonada[n + l][n] / matrizASerEscalonada[n][n]
            matrizASerEscalonada[n + l] = MudaLinha(matrizASerEscalonada[n], matrizASerEscalonada[n + l], razaoPivotamento)
            vetorResposta[n + l] = MudaLinha(vetorResposta[n], vetorResposta[n + l], razaoPivotamento)
            l += 1
        
    return matrizASerEscalonada, vetorResposta
    
def EscalonaSuperior (matrizASerEscalonada, vetorResposta): 
    #ela vai escalonar a diagonal de cima, partindo da diagonal de baixo zerada    
    for n in range(len(matrizASerEscalonada) -1, -1, -1):
        l = 1
        while l <= n: #vai passar p/ pivô anterior
            razaoPivotamento = matrizASerEscalonada[n - l][n] / matrizASerEscalonada[n][n]
            matrizASerEscalonada[n - l] = MudaLinha (matrizASerEscalonada[n], matrizASerEscalonada[n - l], razaoPivotamento)
            vetorResposta[n - l] = MudaLinha(vetorResposta[n], vetorResposta[n - l], razaoPivotamento)
            l += 1

    return matrizASerEscalonada, vetorResposta

def TransformaEscalonadaEmIdentidade (matrizASerEscalonada, vetorResposta): 
   
    for i in range(len(vetorResposta)):
            vetorResposta[i][0] /= matrizASerEscalonada[i][i]
            matrizASerEscalonada[i][i] /= matrizASerEscalonada[i][i]
        
    return matrizASerEscalonada, vetorResposta

In [225]:
def CalculaComposicoesTanque():

    matrizComposicoes = [composicaoW, composicaoX, composicaoY, composicaoZ]
    matrizComposicoesTransposta = TranspoeMatriz(matrizComposicoes)

    massaTotalA = CalculaMassa(concentracaoANoTanque, volumeTanque)
    massaTotalB = CalculaMassa(concentracaoBNoTanque, volumeTanque)
    massaTotalC = CalculaMassa(concentracaoCNoTanque, volumeTanque)
    massaTotalD = CalculaMassa(concentracaoDNoTanque, volumeTanque)

    listaMassaTotalEsteresTanque = [massaTotalA, massaTotalB, massaTotalC, massaTotalD]
    vetorMassaTotalEsteresTanque = TranspoeMatriz([listaMassaTotalEsteresTanque]) # note que eu tô forçando a existência de len(matriz[0]), do contrário acusaria um erro.
    matrizTriangularSuperior, vetorIntermediario1 = EscalonaInferior(matrizComposicoesTransposta, vetorMassaTotalEsteresTanque)
    matrizDiagonal, vetorIntermediario2 = EscalonaSuperior(matrizTriangularSuperior, vetorIntermediario1)
    matrizIdentidade, vetorMassaTotalEsteresCorrentes = TransformaEscalonadaEmIdentidade(matrizDiagonal, vetorIntermediario2)
    vetorVolumeCorrentes = [vetorMassaTotalEsteresCorrentes[i][0]/vetorConcentracaoTotalEsteresCorrentes[i] for i in range(len(vetorMassaTotalEsteresCorrentes))]
    vetorVolumeCorrentes = [vetorVolumeCorrentes[i]/1000 for i in range(len(vetorVolumeCorrentes))] #passando pra m³

    return prettyPrint([vetorVolumeCorrentes], 2)

In [226]:
volumeTanque = 170e3 #L

composicaoW = [0.09, 0.5, 0.4, 0.01]
composicaoX = [0.07, 0.45, 0.45, 0.03]
composicaoY = [0.45, 0.35, 0.2, 0.0]
composicaoZ = [0.25, 0.6, 0.15, 0.0]

vetorConcentracaoTotalEsteresCorrentes = [100, 100, 100, 100]

concentracaoANoTanque = 30.6 #ppm
concentracaoBNoTanque = 47.5 #ppm
concentracaoCNoTanque = 21.6 #ppm
concentracaoDNoTanque = 0.3 #ppm

CalculaComposicoesTanque()

17.0	11.33	71.4	70.27	
